## Consultas phyton implementando HBase

In [ ]:
import happybase
import pandas as pd

In [ ]:
# Bloque principal de ejecución
try:
    # 1. Establecer conexión con HBase
    connection = happybase.Connection('localhost')
    print("Conexión establecida con HBase")

In [ ]:
    # 2. Crear la tabla con las familias de columnas
    table_name = 'red_wine_quality'
    families = {
        'chemistry': dict(),  # datos químicos del vino
        'quality': dict()     # información de calidad
    }

In [ ]:
 # Eliminar la tabla si ya existe
    if table_name.encode() in connection.tables():
        print(f"Eliminando tabla existente - {table_name}")
        connection.delete_table(table_name, disable=True)

In [ ]:
    # Crear nueva tabla
    connection.create_table(table_name, families)
    table = connection.table(table_name)
    print("Tabla 'red_wine_quality' creada exitosamente")

In [ ]:
    # 3. Cargar datos del CSV
    wine_data = pd.read_csv('winequality-red.csv')

In [ ]:
    # Iterar sobre el DataFrame usando el índice
    for index, row in wine_data.iterrows():
        # Generar row key basado en el índice
        row_key = f'wine_{index}'.encode()

In [ ]:
        # Organizar los datos en familias de columnas
        data = {
            b'chemistry:fixed_acidity': str(row['fixed acidity']).encode(),
            b'chemistry:volatile_acidity': str(row['volatile acidity']).encode(),
            b'chemistry:citric_acid': str(row['citric acid']).encode(),
            b'chemistry:residual_sugar': str(row['residual sugar']).encode(),
            b'chemistry:chlorides': str(row['chlorides']).encode(),
            b'chemistry:free_sulfur_dioxide': str(row['free sulfur dioxide']).encode(),
            b'chemistry:total_sulfur_dioxide': str(row['total sulfur dioxide']).encode(),
            b'chemistry:density': str(row['density']).encode(),
            b'chemistry:pH': str(row['pH']).encode(),
            b'chemistry:sulphates': str(row['sulphates']).encode(),
            b'chemistry:alcohol': str(row['alcohol']).encode(),
            b'quality:score': str(row['quality']).encode()
        }
        table.put(row_key, data)
    
    print("Datos cargados exitosamente")

In [ ]:
    # 4. Consultas y Análisis de Datos
    print("\n=== Primeros 3 vinos en la base de datos ===")
    count = 0
    for key, data in table.scan():
        if count < 3:  # Limitamos a 3 para el ejemplo
            print(f"\nVino ID: {key.decode()}")
            print(f"Acidez fija: {data[b'chemistry:fixed_acidity'].decode()}")
            print(f"Acidez volátil: {data[b'chemistry:volatile_acidity'].decode()}")
            print(f"Calidad: {data[b'quality:score'].decode()}")
            count += 1

In [ ]:
    # 5. Análisis del nivel de alcohol por calidad
    print("\n=== Nivel promedio de alcohol por puntuación de calidad ===")
    quality_alcohol = {}
    quality_counts = {}
    
    for key, data in table.scan():
        quality = data[b'quality:score'].decode()
        alcohol = float(data[b'chemistry:alcohol'].decode())
        
        quality_alcohol[quality] = quality_alcohol.get(quality, 0) + alcohol
        quality_counts[quality] = quality_counts.get(quality, 0) + 1
    
    for quality in quality_alcohol:
        avg_alcohol = quality_alcohol[quality] / quality_counts[quality]
        print(f"Calidad {quality}: {avg_alcohol:.2f}% de alcohol promedio")

In [ ]:
   # 6. Análisis de pH por calidad
    print("\n=== pH promedio por puntuación de calidad ===")
    quality_ph = {}
    quality_ph_counts = {}
    
    for key, data in table.scan():
        quality = data[b'quality:score'].decode()
        ph = float(data[b'chemistry:pH'].decode())
        
        quality_ph[quality] = quality_ph.get(quality, 0) + ph
        quality_ph_counts[quality] = quality_ph_counts.get(quality, 0) + 1
    
    for quality in quality_ph:
        avg_ph = quality_ph[quality] / quality_ph_counts[quality]
        print(f"Calidad {quality}: pH promedio de {avg_ph:.2f}")

In [ ]:
    # 7. Top 3 vinos con mayor nivel de ácido sulfúrico
    print("\n=== Top 3 vinos con mayor nivel de ácido sulfúrico ===")
    wines_by_sulphates = []
    for key, data in table.scan():
        wines_by_sulphates.append({
            'id': key.decode(),
            'sulphates': float(data[b'chemistry:sulphates'].decode()),
            'quality': data[b'quality:score'].decode()
        })
    
    for wine in sorted(wines_by_sulphates, key=lambda x: x['sulphates'], reverse=True)[:3]:
        print(f"ID: {wine['id']}")
        print(f"Ácido Sulfúrico: {wine['sulphates']}")
        print(f"Calidad: {wine['quality']}\n")
    
except Exception as e:
    print(f"Error: {str(e)}")
finally:
    # Cerrar la conexión
    connection.close()

## Documentación e interpretación de resultados

### Estructura de la tabla en HBase

Se crea una tabla en HBase llamada red_wine_quality con dos familias de columnas principales:

* #### chemistry:
  Esta familia de columnas almacena características químicas del vino.

* #### quality:
  Esta familia almacena la puntuación de calidad de cada vino.

### Operaciones realizadas en HBase

##### Carga de datos:

Los datos del archivo winequality-red.csv se cargan en HBase. Cada fila corresponde a un vino, y se almacena en HBase, como wine_0, wine_1, etc.

#### Consultas Realizadas:

* Visualización de los primeros 3 Vinos: Se realiza una vista a los 3 primeros vinos verificando el que cargue de manera correcta.

* Promedio de alcohol por calidad: Se calcula el promedio de alcohol respecto a la calidad. Donde evidenciamos que a mayor porcentaje de alcohol, mejor calidad.

* Promedio de pH por calidad: Se calcula el promedio del pH respecto a la calidad. Aquí podemos ver que el comportamiento sugiere que a menor nivel de Ph su calidad aumenta.

* Vinos con mayor nivel de sulfatos: Se muestran los 3 vinos con mayor nivel de sulfatos y podemos evidenciar que estos valores no se encuentran directamente relacionados con la calidad del vino.